In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,10)

In [ ]:
#The path to your bug query log file
bug_log_file = "[your path]\\TFSBug.log"

In [ ]:
#Load the changes from the bug log file
bugs = pd.read_csv(bug_log_file, header=0, delimiter=";", encoding="utf8", error_bad_lines=False, dtype={'WorkItemID': np.int64})
bugs.insert(0, 'id', range(0, len(bugs)))
bugs['ChangeType'] = bugs['ChangeType'].apply(lambda change: change.replace(" ", "").split(","))

In [ ]:
#The path to your TFS branch list
tfs_branch_list = "[your path]\\TFSBranches.list"

In [ ]:
#Remove the branch path from the file paths
branches = pd.read_csv(tfs_branch_list, header=None, delimiter=";", encoding="utf8", error_bad_lines=False) 
branches = branches[0].tolist()

def RemoveBranchFromFile(file):
    for branch in branches:
        file = file.replace(branch, "")
    return file

bugs['File'] = bugs['File'].apply(RemoveBranchFromFile)

In [ ]:
#only look at 'Edit' changes
def IsInList( the_list, item ):
    return not not [ch for ch in the_list if ch == item]

bug_ChangeType_list = bugs['ChangeType'].tolist()
files_edits = bugs[[IsInList(ChangeType_list, 'Edit') for ChangeType_list in bug_ChangeType_list ]]

In [ ]:
#calculate the number of edits per file
files_edits_count = files_edits[['File', 'id']].groupby(['File']).agg(np.size)
files_edits_count = files_edits_count.sort_values('id', ascending=[0])
files_edits_count = files_edits_count.reset_index()
top_change_file_names = files_edits_count.head(30)["File"].tolist()

In [ ]:
files_edits_count.plot.hist()

A histogram showing the distibution of changes over the files

In [ ]:
top_change_file_changes = bugs[bugs['File'].isin(top_change_file_names)]
pd.value_counts(top_change_file_changes['File']).plot.bar()

Most changed files